# Cards for today

- get one ho

In [1]:
from src.utils import load_decision_transformer
import os 
# look for all .pt files recursively in the directory ./artifacts, print their relative paths

for root, dirs, files in os.walk("./artifacts"):
    for file in files:
        if file.endswith(".pt"):
            print(os.path.join(root, file))

./artifacts/MiniGrid-DoorKey-8x8-v0__Dev__1__1673265265:v0/MiniGrid-DoorKey-8x8-v0__Dev__1__1673265265.pt
./artifacts/MiniGrid-DoorKey-8x8-v0__Dev__1__1673268725:v0/MiniGrid-DoorKey-8x8-v0__Dev__1__1673268725.pt
./artifacts/MiniGrid-Dynamic-Obstacles-8x8-v0__MiniGrid-Dynamic-Obstacles-8x8-v0__1__1673629981:v0/MiniGrid-Dynamic-Obstacles-8x8-v0__MiniGrid-Dynamic-Obstacles-8x8-v0__1__1673629981.pt
./artifacts/MiniGrid-Dynamic-Obstacles-8x8-v0__MiniGrid-Dynamic-Obstacles-8x8-v0__1__1674435477:v0/MiniGrid-Dynamic-Obstacles-8x8-v0__MiniGrid-Dynamic-Obstacles-8x8-v0__1__1674435477.pt
./artifacts/MiniGrid-Dynamic-Obstacles-8x8-v0__Dev__1__1673348123:v0/MiniGrid-Dynamic-Obstacles-8x8-v0__Dev__1__1673348123.pt
./artifacts/MiniGrid-Dynamic-Obstacles-8x8-v0__MiniGrid-Dynamic-Obstacles-8x8-v0__1__1674559313:v0/MiniGrid-Dynamic-Obstacles-8x8-v0__MiniGrid-Dynamic-Obstacles-8x8-v0__1__1674559313.pt
./artifacts/MiniGrid-Dynamic-Obstacles-8x8-v0__Dev__1__1673368088:v0/MiniGrid-Dynamic-Obstacles-8x8-v0__

In [1]:
from src.environments import make_env

env_id = "MiniGrid-Dynamic-Obstacles-8x8-v0"
env = make_env(env_id, seed = 1, idx = 0, capture_video=False, run_name = "dev", fully_observed=False, max_steps=30)
env = env()

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/ipywidgets/widgets/widget_selection.py:9: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, Iterable
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/ipywidgets/widgets/widget_selection.py:9: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, Iterable


In [2]:
from minigrid.core.constants import OBJECT_TO_IDX, STATE_TO_IDX, COLOR_TO_IDX

len(OBJECT_TO_IDX) + len(COLOR_TO_IDX) + len(STATE_TO_IDX)

20

# Calibration Plot update to violin 

In [99]:
import pandas as pd 
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

def plot_calibration_statistics(statistics, show_spread=False, CI = 0.95):

    df = pd.DataFrame(statistics)

    # add line to legend
    fig = px.line(df, x="initial_rtg", y="mean_reward", title="Calibration Plot",        template="plotly_white")
    fig.update_layout(
        xaxis_title="Initial RTG",
        yaxis_title="Reward",
        legend_title="",
    )
    fig.add_shape(
        type="line",
        x0=df.initial_rtg.min(), 
        y0=df.initial_rtg.min(),
        x1=df.initial_rtg.max(), 
        y1=df.initial_rtg.max(),
        line=dict(
            color="LightSeaGreen",
            width=4,
            dash="dashdot",
        )
    )

    if show_spread:
        upper = (1-CI)/2
        lower = 1 - upper
        # add 95% CI
        # also show 97.5 percentile and 2.5 percentile
        df["percentile_975"] = df.rewards.apply(lambda x: np.percentile(x,100*upper))
        df["percentile_025"] = df.rewards.apply(lambda x: np.percentile(x,100*lower))
        # shade between 97.5 and 2.5 percentile
        fig.add_trace(
            go.Scatter(
                x=df.initial_rtg,
                y=df.percentile_975,
                mode="lines",
                name=f"{(int(CI*100))}% CI",
                showlegend=False,
                line=dict(color='rgba(0,0,0,0)', width=0.5), # then use light blue fill color
                fillcolor="rgba(0,100,80,0.2)",
            )
        )

        fig.add_trace(
            go.Scatter(
                x=df.initial_rtg,
                y=df.percentile_025,
                mode="lines",
                name=f"{(int(CI*100))}% CI",
                line=dict(color='rgba(0,0,0,0)', width=0.5), # then use light blue fill color
                fillcolor="rgba(0,100,80,0.2)",
                fill="tonexty"
            )   
        )

    return fig 

plot_calibration_statistics(statistics, show_spread=True, CI = 0.95)

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [81]:
df["percentile_975"] 

0    -1.000000
1    -1.000000
2    -1.000000
3    -1.000000
4    -0.100000
5    -0.100000
6    -0.100000
7    -0.100000
8    -0.100000
9     0.000000
10    0.537143
11    0.917475
12    0.920465
13    0.954551
14    0.954551
15    0.961130
16    0.961130
17    0.961130
18    0.961130
19    0.961130
Name: percentile_975, dtype: float64

In [74]:
import pytest 

import warnings
import torch as t
import numpy as np
from src.environments import make_env
from src.decision_transformer.utils import load_decision_transformer
from src.decision_transformer.calibration import calibration_statistics

env_id = "MiniGrid-Dynamic-Obstacles-8x8-v0"
model_path = "models/demo_model_overnight_training.pt"
state_dict = t.load(model_path)
one_hot_encoded = state_dict["state_encoder.weight"].shape[-1] == 980
max_time_steps = state_dict["time_embedding.weight"].shape[0]
env_func = make_env(
    env_id, seed = 1, idx = 0, 
    capture_video=False, run_name = "dev", 
    fully_observed=False, flat_one_hot = one_hot_encoded, max_steps= max_time_steps)
env = env_func()


state_dict = t.load(model_path)
one_hot_encoded = state_dict["state_encoder.weight"].shape[-1] == 980

dt = load_decision_transformer(model_path, env)

warnings.filterwarnings("ignore", category=UserWarning)
statistics = calibration_statistics(
    dt, 
    env_id, 
    env_func=env_func,
    initial_rtg_range=np.linspace(-1, 1, 20),
    trajectories=5,
)

assert statistics is not None
assert len(statistics) == 10


initial_rtg: 1.0: 100%|██████████| 20/20 [00:09<00:00,  2.21it/s]                  


AssertionError: 

In [ ]:
fig = plot_calibration_statistics(statistics)
fig.show()

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [76]:
import plotly.graph_objects as go

df = pd.DataFrame(statistics)
df_expanded = df.explode("rewards")
# fig = px.scatter(df_expanded, x="initial_rtg", y="rewards", title="Calibration Plot", opacity=0.5,
#     template="plotly_white")

# plot mean 


/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [ ]:
# retry the plot above but with violin plot
df = pd.DataFrame(statistics)
df_expanded = df.explode("rewards")
fig = px.violin(df_expanded, y="rewards",  color="initial_rtg", title="Calibration Plot",
    template="plotly_white")
fig.show()

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [62]:
import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
np.random.seed(1)

# 12 sets of normal distributed random data, with increasing mean and standard deviation
data = (np.linspace(1, 2, 12)[:, np.newaxis] * np.random.randn(12, 200) +
            (np.arange(12) + 2 * np.random.random(12))[:, np.newaxis])

colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', 12, colortype='rgb')

fig = go.Figure()
for data_line, color in zip(data, colors):
    fig.add_trace(go.Violin(x=data_line, line_color=color))

fig.update_traces(orientation='h', side='positive', width=3, points=False)
fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False)
fig.show()

df = pd.DataFrame(statistics)
df_expanded = df.explode("rewards")


fig = go.Figure()
colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', len(df.initial_rtg), colortype='rgb')
for initial_rtg, color in zip(df.initial_rtg, colors):
    fig.add_trace(go.Violin(x=df_expanded.rewards[df_expanded.initial_rtg == initial_rtg], line_color=color, name=str(round(initial_rtg, 2))))

fig.update_traces(orientation='h', side='positive', width=3, points=False)
fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False)
fig.show()

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [68]:
# make a ridgeline plot with reward over initial rtg
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go

df = pd.DataFrame(statistics)
df_expanded = df.explode("rewards")
# color contours by density
fig = px.density_contour(df_expanded, x="initial_rtg", y="rewards", title="Calibration Plot",
    template="plotly_white")

# add scatter
fig.add_trace(
    go.Scatter(
        x=df_expanded.initial_rtg,
        y=df_expanded.rewards,
        mode="markers",
        name="Scatter",
        marker=dict(color="blue", size=2)
    )
)
# update x range and y range
fig.update_xaxes(range=[df_expanded.initial_rtg.min(), df_expanded.initial_rtg.max()])
fig.update_yaxes(range=[df_expanded.rewards.min(), df_expanded.rewards.max()])
fig.show()

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [2]:
import wandb
run = wandb.init()
artifact = run.use_artifact('arena-ldn/DecisionTransformerInterpretability/MiniGrid-Dynamic-Obstacles-8x8-v0__MiniGrid-Dynamic-Obstacles-8x8-v0__1__1675306594:v0', type='model')
artifact_dir = artifact.download()

A Jupyter Widget

A Jupyter Widget

wandb:   1 of 1 files downloaded.  


In [3]:
import wandb
run = wandb.init()
artifact = run.use_artifact('arena-ldn/DecisionTransformerInterpretability/MiniGrid-MemoryS13Random-v0__MiniGrid-MemoryS13Random__1__1675311480:v0', type='model')
artifact_dir = artifact.download()

A Jupyter Widget

A Jupyter Widget

wandb:   1 of 1 files downloaded.  


In [4]:
import gymnasium as gym 
import minigrid

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/wandb/sdk/lib/import_hooks.py:246: DeprecationWarning: Deprecated since Python 3.4. Use importlib.util.find_spec() instead.
  loader = importlib.find_loader(fullname, path)
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/wandb/sdk/lib/import_hooks.py:246: DeprecationWarning: Deprecated since Python 3.4. Use importlib.util.find_spec() instead.
  loader = importlib.find_loader(fullname, path)
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/wandb/sdk/lib/import_hooks.py:246: DeprecationWarning: Deprecated since Python 3.4. Use importlib.util.find_spec() instead.
  loader = importlib.find_loader(fullname, path)


In [9]:
minigrid_envs = [i for i in gym.envs.registry.keys() if "MiniGrid" in i]
minigrid_envs

['MiniGrid-BlockedUnlockPickup-v0',
 'MiniGrid-LavaCrossingS9N1-v0',
 'MiniGrid-LavaCrossingS9N2-v0',
 'MiniGrid-LavaCrossingS9N3-v0',
 'MiniGrid-LavaCrossingS11N5-v0',
 'MiniGrid-SimpleCrossingS9N1-v0',
 'MiniGrid-SimpleCrossingS9N2-v0',
 'MiniGrid-SimpleCrossingS9N3-v0',
 'MiniGrid-SimpleCrossingS11N5-v0',
 'MiniGrid-DistShift1-v0',
 'MiniGrid-DistShift2-v0',
 'MiniGrid-DoorKey-5x5-v0',
 'MiniGrid-DoorKey-6x6-v0',
 'MiniGrid-DoorKey-8x8-v0',
 'MiniGrid-DoorKey-16x16-v0',
 'MiniGrid-Dynamic-Obstacles-5x5-v0',
 'MiniGrid-Dynamic-Obstacles-Random-5x5-v0',
 'MiniGrid-Dynamic-Obstacles-6x6-v0',
 'MiniGrid-Dynamic-Obstacles-Random-6x6-v0',
 'MiniGrid-Dynamic-Obstacles-8x8-v0',
 'MiniGrid-Dynamic-Obstacles-16x16-v0',
 'MiniGrid-Empty-5x5-v0',
 'MiniGrid-Empty-Random-5x5-v0',
 'MiniGrid-Empty-6x6-v0',
 'MiniGrid-Empty-Random-6x6-v0',
 'MiniGrid-Empty-8x8-v0',
 'MiniGrid-Empty-16x16-v0',
 'MiniGrid-Fetch-5x5-N2-v0',
 'MiniGrid-Fetch-6x6-N2-v0',
 'MiniGrid-Fetch-8x8-N3-v0',
 'MiniGrid-FourRoom

In [13]:
from src.decision_transformer.offline_dataset import TrajectoryLoader
from src.environments import make_env
import torch as t
trajectory_loader = TrajectoryLoader("trajectories/MiniGrid-MemoryS13Random-v011a50070-9f4f-4524-a55e-43282689e5eb.xz")

{'args': {'exp_name': 'MiniGrid-MemoryS13Random-v0', 'seed': 1, 'cuda': True, 'track': True, 'wandb_project_name': 'PPO-MiniGrid', 'wandb_entity': None, 'capture_video': True, 'env_id': 'MiniGrid-MemoryS13Random-v0', 'total_timesteps': 200000, 'learning_rate': 0.00025, 'num_envs': 30, 'num_steps': 64, 'gamma': 0.99, 'gae_lambda': 0.95, 'num_minibatches': 30, 'update_epochs': 4, 'clip_coef': 0.4, 'ent_coef': 0.2, 'vf_coef': 0.5, 'max_grad_norm': 2.0, 'max_steps': 500, 'one_hot_obs': True, 'trajectory_path': 'trajectories/MiniGrid-MemoryS13Random-v011a50070-9f4f-4524-a55e-43282689e5eb.xz', 'fully_observed': False}, 'time': 1675310599.489821}


In [35]:
import plotly.express as px 
s, a, r, d, rtg, timesteps, mask = trajectory_loader.get_batch(1)
fig = px.imshow(s[0, :, :, :, 0].transpose(-1,-2), animation_frame=0, range_color=[0, 1])
fig.show()

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [21]:
s.shape

torch.Size([1, 100, 7, 7, 20])

In [36]:

mask

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=torch.float64)

In [38]:
a

tensor([[-10., -10., -10., -10., -10., -10., -10., -10., -10., -10., -10., -10.,
         -10., -10., -10., -10., -10., -10., -10., -10., -10., -10., -10., -10.,
         -10., -10., -10., -10., -10., -10., -10., -10., -10., -10., -10., -10.,
         -10., -10., -10., -10., -10., -10., -10., -10., -10., -10., -10., -10.,
         -10., -10., -10., -10., -10., -10., -10., -10., -10., -10., -10., -10.,
         -10., -10.,   1.,   4.,   1.,   0.,   2.,   3.,   4.,   0.,   5.,   0.,
           2.,   4.,   6.,   2.,   3.,   1.,   6.,   1.,   6.,   2.,   3.,   1.,
           3.,   0.,   5.,   0.,   5.,   6.,   5.,   2.,   4.,   5.,   0.,   5.,
           2.,   6.,   0.,   2.]])

In [40]:
r.squeeze()

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.7498])